Solana's price is very volatile and trading volume is not always consistent. So the VWAP we are calculating to approximate the Realized Price is going to go up and down and be very noisy. These sudden price jumps can heavily skew the VWAP and make the realized calculation unreliable, so we apply a rolling average instead to smooth it out. Instead of using the raw VWAP we take an average of the last 10 points (hourly or average) which gives a better idea of the overall trend than current instant.

However, if the data shows low volatility and stable price movement, smoothing can unnecessarily blur small but meaningful changes. Ideally, smoothing should be applied conditionally—only when volatility exceeds a certain threshold. Due to time constraints, this conditional smoothing has not been implemented here, but it is recommended for future improvements.

In [1]:
import requests
import pandas as pd
import plotly.graph_objs as go

def get_solana_supply():
    """
    Fetch current circulating supply for Solana from CoinGecko
    """
    url = "https://api.coingecko.com/api/v3/coins/solana"
    resp = requests.get(url)
    data = resp.json()
    return data["market_data"]["circulating_supply"]

def fetch_solana_market_data(days, interval=None):
    """
    Fetch Solana price and volume data from CoinGecko market_chart endpoint
    """
    url = "https://api.coingecko.com/api/v3/coins/solana/market_chart"
    params = {
        "vs_currency": "usd",
        "days": days
    }
    if interval:
        params["interval"] = interval
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code} - {response.text}")
    data = response.json()
    if "prices" not in data:
        raise Exception(f"'prices' key missing in response: {data}")
    return data

def compute_mvrv(data, sol_supply, smoothing_window=10):
    """
    Compute MVRV ratio for Solana, with VWAP smoothing.
    """
    # Extracting prices and volume from the JSON data
    prices = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
    volumes = pd.DataFrame(data["total_volumes"], columns=["timestamp", "volume"])

    # Add a new datetime column that converts the milliseconds timestamp into human readable format
    prices["datetime"] = pd.to_datetime(prices["timestamp"], unit="ms")
    volumes["datetime"] = pd.to_datetime(volumes["timestamp"], unit="ms")

    # Merging Price and Volume by Time
    df = pd.merge(prices[["datetime", "price"]], volumes[["datetime", "volume"]], on="datetime")

    # Calculate value = price * volume
    df["value_contrib"] = df["price"] * df["volume"]

    # Calculate cumulative value
    df["cum_value"] = df["value_contrib"].cumsum()

    # Calculate cumulative volume
    df["cum_volume"] = df["volume"].cumsum()

    # Calculates VWAP (Volume-Weighted Average Price) = cumulative value / cumulative volume.
    df["vwap"] = df["cum_value"] / df["cum_volume"]

    # Apply rolling average smoothing to the VWAP to reduce noise.
    df["vwap"] = df["vwap"].rolling(window=smoothing_window, min_periods=1).mean()

    # Total number of Solana coins in circulation
    df["circulating_supply"] = sol_supply

    # Calculate Realized Value
    df["realized_value"] = df["vwap"] * df["circulating_supply"]
    # Calculate Market Value
    df["market_value"] = df["price"] * df["circulating_supply"]

    # Calculate MVRV = Market Value/Realized Value
    df["mvrv_ratio"] = df.apply(
        lambda row: (row["market_value"] / row["realized_value"]) if row["realized_value"] != 0 else None, axis=1
    )

    return df[["datetime", "price", "volume", "vwap", "circulating_supply", "realized_value", "market_value", "mvrv_ratio"]]

def main():
    print("Fetching SOL circulating supply...")
    sol_supply = get_solana_supply()
    print(f"SOL Supply: {sol_supply:,.0f} coins")

    print("Fetching daily data (max 365 days)...")
    daily_data = fetch_solana_market_data(days=365)
    daily_mvrv = compute_mvrv(daily_data, sol_supply)
    daily_mvrv.to_csv("sol_daily_mvrv.csv", index=False)
    print("Saved daily MVRV to sol_daily_mvrv.csv")

    print("Fetching hourly data (last 90 days)...")
    hourly_data = fetch_solana_market_data(days=90)
    hourly_mvrv = compute_mvrv(hourly_data, sol_supply)
    hourly_mvrv.to_csv("sol_hourly_mvrv.csv", index=False)
    print("Saved hourly MVRV to sol_hourly_mvrv.csv")

if __name__ == "__main__":
    main()


Fetching SOL circulating supply...
SOL Supply: 534,453,709 coins
Fetching daily data (max 365 days)...
Saved daily MVRV to sol_daily_mvrv.csv
Fetching hourly data (last 90 days)...
Saved hourly MVRV to sol_hourly_mvrv.csv


In [2]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Load Solana daily MVRV CSV
df = pd.read_csv("sol_daily_mvrv.csv", parse_dates=["datetime"])

# Create 2 plots Market Price vs Realized Price on top, MVRV ratio below
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.1,
                    subplot_titles=("Daily Market Price & Realized Price over Time", "Daily MVRV Ratio over Time"))

fig.add_trace(go.Scatter(x=df["datetime"], y=df["price"], mode="lines", name="Market Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["vwap"], mode="lines", name="Realized (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["mvrv_ratio"], mode="lines", name="MVRV Ratio"), row=2, col=1)

# Horizontal line at MVRV=1 (fair value)
fig.add_shape(type="line", x0=df["datetime"].min(), x1=df["datetime"].max(),
              y0=1, y1=1, yref='y2', line=dict(color="Red", dash="dash"))

fig.update_layout(height=600, width=900, title_text="Daily Solana MVRV Dashboard", template="plotly_white")
fig.update_yaxes(range=[0.5, df["mvrv_ratio"].max() * 1.1], row=2, col=1)

fig.show()


In [3]:
# Load Solana hourly MVRV CSV
df = pd.read_csv("sol_hourly_mvrv.csv", parse_dates=["datetime"])

# Create 2 plots Market Price vs Realized Price on top, MVRV ratio below
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Hourly Market Price & Realized Price over Time", "Hourly MVRV Ratio over Time")
)

fig.add_trace(go.Scatter(x=df["datetime"], y=df["price"], mode="lines", name="Market Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["vwap"], mode="lines", name="Realized Price (USD)"), row=1, col=1)
fig.add_trace(go.Scatter(x=df["datetime"], y=df["mvrv_ratio"], mode="lines", name="MVRV Ratio"), row=2, col=1)

# Horizontal line at MVRV = 1 (fair value)
fig.add_shape(type="line",
              x0=df["datetime"].min(), x1=df["datetime"].max(),
              y0=1, y1=1,
              yref='y2',
              line=dict(color="Red", dash="dash"),
              row=2, col=1)

fig.update_layout(height=600, width=900, title_text="Hourly Solana MVRV Dashboard", template="plotly_white")
fig.update_yaxes(range=[0.5, df["mvrv_ratio"].max() * 1.1], row=2, col=1)

fig.show()
